# Combining genotype info

### Load data
We need to combine the genotype info with the AngioTool data.


Start by loading the Excel spreadsheet with select columns (we don't care about the date, time, etc.) into a `pandas` DataFrame. 

In [1]:
import numpy as np
import pandas as pd

#tweaked_results = './processed/tweaked/TweakedResults11-6-2017.xls'
tweaked_results = '../vessels_analysis/2018-03-28_concatenated_results.xls'
data = pd.read_excel(tweaked_results,
                     usecols = "A, N, P, O, Q, S")
#                     usecols = "A, G, N, O, Q, S")
# These are the Excel columns that correspond to 
# the desired AngioTool output parameters

#pd.options.display.float_format = '{:,.2f}'.format

Confirm you loaded the correct thing by looking at the header. You can always look at a Dataframe's first five entries through `df.head()`

In [2]:
data.head()

,Image Name,Vessels percentage area,Total Number of Junctions,Junctions density,Total Vessels Length,Total Number of End Points
0,42-2-1_left.tif,43.987325,118,142.170990,15.046215,65
1,42-2-1_right.tif,39.556208,109,131.984783,13.302744,50
2,42-2-1_top.tif,39.675365,89,109.754133,12.119949,43
3,42-2-2_bottom.tif,38.355378,116,141.146078,14.567042,46
4,42-2-2_left.tif,49.424402,143,172.670391,15.450351,42


In [3]:
data["Image Name"] = data["Image Name"].str.split(".tif", expand=True)
data.head()

,Image Name,Vessels percentage area,Total Number of Junctions,Junctions density,Total Vessels Length,Total Number of End Points
0,42-2-1_left,43.987325,118,142.170990,15.046215,65
1,42-2-1_right,39.556208,109,131.984783,13.302744,50
2,42-2-1_top,39.675365,89,109.754133,12.119949,43
3,42-2-2_bottom,38.355378,116,141.146078,14.567042,46
4,42-2-2_left,49.424402,143,172.670391,15.450351,42


### Create unique IDs for each animal
We need to parse the Image Name column in order to get a unique ID for each animal. The unique ID will be used for pairing genotypes using the Genotyping Key document.

I'm using list comprehensions to get Litter/ID info from the Image Name. Both are added to our `data` DataFrame.

In [4]:
data["Image Name"][0].split(".tif")[0]

'42-2-1_left'

In [5]:
data['Litter'] =   [x[:2] for x in data['Image Name']]
data['AnimalID'] = [x[:4] for x in data['Image Name']]

In [6]:
data.head()

,Image Name,Vessels percentage area,Total Number of Junctions,Junctions density,Total Vessels Length,Total Number of End Points,Litter,AnimalID
0,42-2-1_left,43.987325,118,142.170990,15.046215,65,42,42-2
1,42-2-1_right,39.556208,109,131.984783,13.302744,50,42,42-2
2,42-2-1_top,39.675365,89,109.754133,12.119949,43,42,42-2
3,42-2-2_bottom,38.355378,116,141.146078,14.567042,46,42,42-2
4,42-2-2_left,49.424402,143,172.670391,15.450351,42,42,42-2


### Load genotyping key
Once again, we'll use the `read_excel` tool found within the `pandas` library.

In [7]:
key = pd.read_csv('../genotypingkey_P7.csv')
key

,Litter,Animal,AnimalID,Type
0,42,2,42-2,mut
1,42,3,42-3,mut
2,42,4,42-4,wt
3,42,5,42-5,wt
4,42,6,42-6,het
5,43,0,43-0,wt
6,43,1,43-1,wt
7,43,2,43-2,mut
8,43,3,43-3,wt
9,43,4,43-4,mut


### Obtain entire dataset
We'll merge the two dataframes by using the `merge` function from the `pandas` library. Notice how `AnimalID` is used to match.

We can also drop unneeded columns such as `Image Name`, `ID`, and the duplicated `Litter_y`.

In [8]:
merged_inner = pd.merge(left = data, right = key,
                        how = 'inner',
                        left_on =  'AnimalID',
                        right_on = 'AnimalID'
                       )
## Drop unneeded columns
merged_inner.drop('Litter_y',         axis = 1, inplace = True)
#merged_inner.drop('ID',               axis = 1, inplace = True)
merged_inner.drop('Image Name',       axis = 1, inplace = True)
#merged_inner.drop('Vessel Thickness', axis = 1, inplace = True)

merged_inner.rename(columns = {'Litter_x':'Litter'},
                    inplace = True)


#pd.options.display.float_format = '{:,.2f}'.format

## Show first five observations
merged_inner.head()

,Vessels percentage area,Total Number of Junctions,Junctions density,Total Vessels Length,Total Number of End Points,Litter,AnimalID,Animal,Type
0,43.987325,118,142.170990,15.046215,65,42,42-2,2,mut
1,39.556208,109,131.984783,13.302744,50,42,42-2,2,mut
2,39.675365,89,109.754133,12.119949,43,42,42-2,2,mut
3,38.355378,116,141.146078,14.567042,46,42,42-2,2,mut
4,49.424402,143,172.670391,15.450351,42,42,42-2,2,mut


In [9]:
## Round floats to 3 decimal points

merged_inner['Vessel Density'] = (merged_inner['Vessels percentage area']
                                           .apply(lambda x: round(x, 3)))
merged_inner['Vessel Length'] = (merged_inner['Total Vessels Length']
                                        .apply(lambda x: round(x, 3)))
merged_inner['Junction Density'] = (merged_inner['Junctions density']
                                        .apply(lambda x: round(x, 3)))

In [10]:
df = merged_inner[["Total Number of End Points",
                     "Litter", "AnimalID", "Animal","Type","Vessel Density",
                     "Vessel Length", "Junction Density", "Total Number of Junctions"]].copy()
df.head()

,Total Number of End Points,Litter,AnimalID,Animal,Type,Vessel Density,Vessel Length,Junction Density,Total Number of Junctions
0,65,42,42-2,2,mut,43.987,15.046,142.171,118
1,50,42,42-2,2,mut,39.556,13.303,131.985,109
2,43,42,42-2,2,mut,39.675,12.120,109.754,89
3,46,42,42-2,2,mut,38.355,14.567,141.146,116
4,42,42,42-2,2,mut,49.424,15.450,172.670,143


In [11]:
## Save to a csv file
## 
df.to_csv("../UnblindedAngioToolDataP7.csv", index = False)